**A- Bibliothèques, concepts et initialisation**

1- Pourquoi ces bibliothèques ?

- ee: API python de GEE qui donne accès aux collections d'images, FeatureCollections, fonctions de réduction, export, etc. C'est la liaison principale avec GEE.

- geopandas: manipulation des géo-données vectorielles côté client (lecture de GeoJSON/Shapefiles locaux, reprojection, calculs de surfaces, jointures attributaires). Utile pour analyses fines hors du serveur GEE.

- matplotlib: affichage statique des images

- rasterio: lire/écrire des fichiers raster (.tif).

- folium ou geemap: visualisation interactive dans un notebook. geemap fournit des fonctions utiles pour afficher directement des objets Earth Engine.

- os, json, requests: utilitaires (gestion de fichiers, conversions, récupération d'images via URL).

2- Initialisation, authentification et connexion

- Activer l'API Earth Engine pour notre compte Google

In [24]:
# Imports et initialisation
import ee
import os
import json
import geopandas as gpd
import matplotlib.pyplot as plt


# Authentification
ee.Authenticate(auth_mode='notebook')

# Initialisation en précisant l'ID du projet
ee.Initialize(project='ee-leslyenkwa')

# visualisation interactive
import geemap
Map = geemap.Map(center=[7.5, 2.3], zoom=6)

3- Notions fondamentales: raster, vecteur, projection

Un **raster** est une représentation matricielle d'une surface continue où chaque cellule (pixel) contient une valeur liée à une variable (ex : population estimée, indice, température).

Le raster est défini par :

- résolution spatiale (taille d'un pixel, ex. 100 m) ;

- étendue spatiale (bounding box) ;

- projection / CRS ;

- nombre de bandes.

**B- Importation des données (assets et rasters) depuis GEE**

- FeatureCollection pointe vers un asset vectoriel stocké dans le projet.
- ImageCollection est filtrée pour obtenir l'image souhaitée (WorldPop 2020)
- getInfo() permets la récupération des métadonnées

In [25]:
# Récupération des assets vecteurs
village = ee.FeatureCollection("projects/ee-leslyenkwa/assets/village")
commune = ee.FeatureCollection("projects/ee-leslyenkwa/assets/commune")
departement = ee.FeatureCollection("projects/ee-leslyenkwa/assets/departement")
pays = ee.FeatureCollection("projects/ee-leslyenkwa/assets/pays")


# Petit contrôle
print('Pays — premier feature :', pays.first().getInfo())

Pays — premier feature : {'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[1.9487520193215102, 6.2998580639665285], [1.947918173247689, 6.299858077553564], [1.947918173247689, 6.300139028859504], [1.9465269229568918, 6.300139000306086], [1.9465269229568918, 6.299858129856313], [1.946250453632985, 6.2998581008501064], [1.946250453632985, 6.299581608800146], [1.9465269229568918, 6.299581637845916], [1.9465269229568918, 6.299305147063388], [1.9473607428165671, 6.299305136026305], [1.9473607428165671, 6.299581626689028], [1.9487520193215102, 6.299581653334783], [1.9487520193215102, 6.2998580639665285]]], [[[1.9295823078125394, 6.294306550250358], [1.9295823078125394, 6.294030080475532], [1.929858822093872, 6.294030037881926], [1.929858822093872, 6.293749111522758], [1.9323603556872664, 6.293749160852131], [1.9323603556872664, 6.294030086849405], [1.9326368127718847, 6.294030043002088], [1.9326368127718847, 6.294306512341599], [1.9320839003316765, 6.29430651891094]

In [26]:
# Rasters
# Population WorldPop 2020 (100m)
worldpop = ee.ImageCollection("WorldPop/GP/100m/pop") \
.filter(ee.Filter.eq('country', 'BEN')) \
.filter(ee.Filter.eq('year', 2020)) \
.first()

# Malaria
tiff_malaria = ee.Image("projects/ee-leslyenkwa/assets/202508_Global_Pf_Parasite_Rate_BEN_2020")

In [27]:
print('WorldPop image info keys :', worldpop.getInfo().keys())
print('Malaria image info keys :', tiff_malaria.getInfo().keys())

WorldPop image info keys : dict_keys(['type', 'bands', 'version', 'id', 'properties'])
Malaria image info keys : dict_keys(['type', 'bands', 'version', 'id', 'properties'])


**C- Affichge des propriétés**

In [32]:
# Projection nominale
wp_proj = worldpop.projection().getInfo()
mal_proj = tiff_malaria.projection().getInfo()
print("WorldPop projection :", wp_proj)
print("Malaria projection :", mal_proj)




WorldPop projection : {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Malaria projection : {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.041666668869465705, 0, 0.7916729666719302, 0, -0.04165069164673973, 12.390784591253556]}


In [33]:
# Échelle nominale d'un pixel
wp_scale = worldpop.projection().nominalScale().getInfo()
mal_scale = tiff_malaria.projection().nominalScale().getInfo()
print("WorldPop pixel ≈", wp_scale, "m")
print("Malaria pixel ≈", mal_scale, "m")

WorldPop pixel ≈ 111319.49079327357 m
Malaria pixel ≈ 4637.42298818532 m


Projections

- WorldPop : EPSG:4326

11 319 m ≈ 111 km	:Très grossier, en fait c’est la projection *EPSG:4326 qui* fait que le pixel géographique est énorme si on ne reprojettes pas.

- Malaria (PfPR) : EPSG:4326

4 637 m ≈ 4.6 km: Plus fin, résolution plus réaliste pour le Bénin

*Les deux sont en coordonnées géographiques (latitude/longitude).

*Donc pas de problème de CRS pour la superposition.

Remarque importante : même si le CRS est le même, la résolution (taille des pixels) est très différente.



In [51]:
print("Bandes malaria :", tiff_malaria.bandNames().getInfo())

Bandes malaria : ['b1', 'b2', 'b3', 'b4']


In [55]:
# WorldPop : première bande
worldpop_band = worldpop.select(0)

# Malaria : première bande
malaria_band = tiff_malaria.select(0)

malaria_band
worldpop_band

Output hidden; open in https://colab.research.google.com to view.

In [50]:
# --- WorldPop ---
print("WorldPop : bandes =", worldpop_band.bandNames().getInfo())
print("WorldPop : projection =", worldpop_band.projection().getInfo())
print("WorldPop : échelle (m) =", worldpop_band.projection().nominalScale().getInfo())
print("WorldPop : propriétés =", worldpop_band.propertyNames().getInfo())

# --- Malaria ---
print("Malaria : bandes =", malaria_band.bandNames().getInfo())
print("Malaria : projection =", malaria_band.projection().getInfo())
print("Malaria : échelle (m) =", malaria_band.projection().nominalScale().getInfo())
print("Malaria : propriétés =", malaria_band.propertyNames().getInfo())


WorldPop : bandes = ['population']
WorldPop : projection = {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
WorldPop : échelle (m) = 111319.49079327357
WorldPop : propriétés = ['system:footprint', 'system:bands', 'system:band_names']
Malaria : bandes = ['b1']
Malaria : projection = {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.041666668869465705, 0, 0.7916729666719302, 0, -0.04165069164673973, 12.390784591253556]}
Malaria : échelle (m) = 4637.42298818532
Malaria : propriétés = ['system:footprint', 'system:version', 'system:id', 'system:asset_size', 'system:bands', 'system:band_names']


Nombre de bandes

- WorldPop : 1 bande → population

- Malaria : 1 bande → PfPR₂₋₁₀ (proportion infectée)

Projection (CRS)

- Les deux en EPSG:4326 → latitude/longitude. Ce qui est compatible pour superposition avec les shapefiles du Bénin

Échelle / taille d’un pixel

- WorldPop : ~111 km → très grossier → nécessite un resampling si tu veux comparer avec Malaria

- Malaria : ~4,6 km → assez fin pour les analyses communales

Propriétés

- Contiennent les métadonnées système (date, footprint, nom des bandes, taille de l’asset)

Transformation

- WorldPop : [1,0,0,0,1,0] → pixel 1° × 1° (~111 km)

- Malaria : [0.04166..., 0, 0.79..., 0, -0.04165..., 12.39...] → pixel ~0.0416° (~4,6 km), origine et alignement spécifiques

Ce qui est important pour s’assurer que les rasters s’alignent correctement sur le shapefile

**D- Visualisation**

Principe : Earth Engine ne renvoie pas directement des objets Matplotlib.

- getThumbURL / getDownloadURL pour obtenir une image(s) côté client et l'afficher avec matplotlib.

- geemap/folium pour ajouter des couches interactives.

- Exporter en GeoTIFF/GeoJSON et lire localement avec rasterio/geopandas.

1- Visualiser le raster (WorldPop)

In [21]:
Map = geemap.Map(center=[9.3, 2.3], zoom=7)
Map

Map(center=[9.3, 2.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [66]:
commune_features = commune.getInfo()['features']

for f in commune_features:
    geom = f['geometry']
    name = f['properties']['NAME_2']

    if geom['type'] == 'Polygon':
        coords = geom['coordinates'][0]
    elif geom['type'] == 'MultiPolygon':
        coords = geom['coordinates'][0][0]  # premier polygone, premier anneau
    else:
        continue

    lon = sum([c[0] for c in coords]) / len(coords)
    lat = sum([c[1] for c in coords]) / len(coords)

    folium.map.Marker(
        [lat, lon],
        icon=folium.DivIcon(html=f'<div style="font-size:10px;color:black">{name}</div>')
    ).add_to(m)

m

In [ ]:
# Affichage des limites communales
Map.addLayer(commune.style(color='blue', fillColor='00000000', width=1.2), {}, "Communes")
Map

Map(bottom=8066.0, center=[9.318990192397917, 3.2735528582109112], controls=(WidgetControl(options=['position'…

In [40]:
# Carte centrée sur le Bénin
Map = geemap.Map(center=[9.5, 2.3], zoom=7)

# Paramètres de visualisation WorldPop
vis_wp = {
    "min": 0,
    "max": 6.75,
    "palette": ["#f7fcfd", "#e5f5f9", "#ccece6", "#99d8c9", "#66c2a4", "#41ae76", "#238b45", "#005824"]
}

# Paramètres de visualisation Malaria PfPR
vis_malaria = {
    "min": 0,
    "max": 1,
    "palette": ['#ffffcc', '#fd8d3c', '#bd0026']
}


In [41]:
Map = geemap.Map(center=[9.5, 2.3], zoom=7)
Map.addLayer(worldpop, vis_wp, "WorldPop 2020")
Map


Map(center=[9.5, 2.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [42]:
# Ajout des limites communales
Map.addLayer(worldpop, vis_wp, f"WorldPop {YEAR_WP}")

Map.addLayer(commune.style(color='black', fillColor='00000000'), {}, "Communes")

Map

Map(bottom=15815.0, center=[9.5, 2.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

2- Visauliser ler raster Malaria

In [45]:
# Sélection de la première bande avant visualisation
malaria_vis = tiff_malaria.select(0)

vis_malaria = {
    "min": 0,
    "max": 1,
    "palette": ['#ffffcc', '#fd8d3c', '#bd0026']
}

In [46]:
Map = geemap.Map(center=[9.5, 2.3], zoom=7)
Map.addLayer(malaria_vis, vis_malaria, "PfPR₂₋₁₀ 2020")
Map

Map(center=[9.5, 2.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [56]:
# Reprojection de WorldPop pour qu’il ait la même taille de pixel que Malaria
worldpop_resampled = worldpop.resample('bilinear').reproject(crs='EPSG:4326', scale=4637)


In [57]:
# Clip au niveau des communes
worldpop_clip = worldpop_resampled.clip(commune.geometry())
malaria_clip = tiff_malaria.select(0).clip(commune.geometry())


3-Visualiser les deux rasters ensemble

In [58]:
# Visualisation après sélection de la bande

vis_wp = {"min":0, "max":6.75, "palette":["#f7fcfd","#e5f5f9","#ccece6","#99d8c9","#66c2a4","#41ae76","#238b45","#005824"]}
vis_malaria = {"min":0, "max":1, "palette":["#ffffcc","#fd8d3c","#bd0026"]}

Map = geemap.Map(center=[9.5,2.3], zoom=7)
Map.addLayer(worldpop_clip.select(0), vis_wp, "WorldPop 2020")
Map.addLayer(malaria_clip, vis_malaria, "PfPR₂₋₁₀ 2020")
Map.addLayer(commune.style(color='black', fillColor='00000000'), {}, "Communes")
Map


Map(center=[9.5, 2.3], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

4- Quelques statistiques

In [59]:
# Vérifier la structure des propriétés de la première commune
print(commune.first().getInfo())


{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[2.6319665465992967, 6.506635501006941], [2.6418077895185013, 6.500816424189276], [2.64291361904367, 6.497249144081565], [2.6447508175283145, 6.491318492529168], [2.645656034821903, 6.488411148481991], [2.647243408199827, 6.483278749895916], [2.6511540786315133, 6.472340624611146], [2.6531428374416786, 6.466829134417708], [2.652313470471964, 6.447913644973607], [2.661044383672873, 6.445260507166203], [2.6783857901945924, 6.445510179470505], [2.6924230833337965, 6.448965991641631], [2.6978275034870713, 6.450129838412254], [2.7102104294737503, 6.458802783171295], [2.7135279958469343, 6.461121492194361], [2.7135056908211794, 6.4601449170164065], [2.7143082990865257, 6.4623210226730325], [2.7157797958830363, 6.466289586208934], [2.717429717709072, 6.470748703609053], [2.7178533228361412, 6.473495477917447], [2.718330423786013, 6.476607947949591], [2.7193025471571617, 6.482935384917266], [2.7195121193204925, 6.488174831036

In [67]:
# Fonction pour calculer stats
def zonal_stats(image, features, scale):
    stats = image.reduceRegions(
        collection=features,
        reducer=ee.Reducer.mean()
                .combine(ee.Reducer.min(), '', True)
                .combine(ee.Reducer.max(), '', True)
                .combine(ee.Reducer.stdDev(), '', True),
        scale=scale
    )
    return stats

# Échelle : choisir la plus fine (PfPR)
scale = 4637  # m

# Calcul stats
wp_stats = zonal_stats(worldpop_clip, commune, scale)
malaria_stats = zonal_stats(malaria_clip, commune, scale)


In [75]:
def calculer_statistiques():
    """Calcule des statistiques descriptives complètes"""

    print("\n STATISTIQUES DESCRIPTIVES ")

    # Statistiques par département
    stats_departements_pop = worldpop.reduceRegions(
        collection=departement,
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax().combine(
                reducer2=ee.Reducer.stdDev(),
                sharedInputs=True
            ),
            sharedInputs=True
        ),
        scale=100
    )

    stats_departements_malaria = malaria.reduceRegions(
        collection=departement,
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax().combine(
                reducer2=ee.Reducer.stdDev(),
                sharedInputs=True
            ),
            sharedInputs=True
        ),
        scale=5000
    )

    print("Statistiques calculées par département")

    # Affichage des résultats
    print("\n--- POPULATION PAR DÉPARTEMENT ---")
    features_pop = stats_departements_pop.getInfo()['features']
    for feature in features_pop:
        props = feature['properties']
        print(f"Département : {props.get('NAME_1', 'N/A')}")
        print(f"  Population moyenne: {props.get('mean', 0):.2f}")
        print(f"  Min: {props.get('min', 0):.2f}, Max: {props.get('max', 0):.2f}")
        print(f"  Écart-type: {props.get('stdDev', 0):.2f}")

    return stats_departements_pop, stats_departements_malaria

stats_pop_dept, stats_malaria_dept = calculer_statistiques()


 STATISTIQUES DESCRIPTIVES 
Statistiques calculées par département

--- POPULATION PAR DÉPARTEMENT ---
Département : Kouffo
  Population moyenne: 3.20
  Min: 0.22, Max: 24.89
  Écart-type: 2.66
Département : Alibori
  Population moyenne: 0.38
  Min: 0.07, Max: 13.95
  Écart-type: 0.48
Département : Atakora
  Population moyenne: 0.42
  Min: 0.07, Max: 9.53
  Écart-type: 0.43
Département : Atlantique
  Population moyenne: 5.31
  Min: 0.43, Max: 105.63
  Écart-type: 8.77
Département : Borgou
  Population moyenne: 0.56
  Min: 0.04, Max: 29.32
  Écart-type: 1.20
Département : Collines
  Population moyenne: 0.52
  Min: 0.11, Max: 6.35
  Écart-type: 0.48
Département : Donga
  Population moyenne: 0.54
  Min: 0.11, Max: 10.94
  Écart-type: 0.61
Département : Littoral
  Population moyenne: 108.11
  Min: 6.39, Max: 184.69
  Écart-type: 51.33
Département : Mono
  Population moyenne: 3.38
  Min: 0.61, Max: 32.57
  Écart-type: 2.70
Département : Ouémé
  Population moyenne: 9.96
  Min: 0.64, Max: 12